## PortQry Log Summariser

This notebook uses a Large Language Model (LLM) to analyse PortQry firewall test log files.  
Given a raw PortQry log, the LLM:

- Parses each tested host, port, protocol, and status (LISTENING, NOT LISTENING, FILTERED, NO RESPONSE).
- Classifies each check as a passed or failed firewall rule.
- Generates a human-readable markdown summary of the results.
- Produces CSV output for further analysis or reporting.

In [ ]:
#imports 

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
# Reading and display portqry file contents
log_filename = "portqry_results.log"

# Read the file content into a string
with open(log_filename, "r", encoding="utf-8") as f:
    log_contents = f.read()

print(log_contents)

In [ ]:
# system and user prompts 

system_prompt = """
You are a highly reliable assistant that analyzes firewall-test log files generated using PortQry.

Your responsibilities:

1. Parse the log contents accurately, identifying:
   - target host
   - each tested port
   - protocol (TCP/UDP)
   - PortQry status (LISTENING, NOT LISTENING, FILTERED, or NO RESPONSE)

2. Classify firewall rule outcomes:
   - Passed → LISTENING
   - Failed → NOT LISTENING, FILTERED, or NO RESPONSE

3. Generate two outputs:
   - A markdown summary of the results.
   - A CSV file with passed/failed results (Passed Rules, Failed Rules) with the following columns:
         Target,IP,Port,Protocol,Status(LISTENING/NOT LISTENING/FILTERED/NO RESPONSE),Result(Passed/Failed)
     Please ensure that the CSV output matches the CSV format with each rule in a separate line. 

4. Ensure the analysis is accurate, structured, and suitable for automated processing.

5. Please add the following headers in the response content:
   -> ---SUMMARY_START--- and ---SUMMARY_END--- to wrap summary text.
   -> ---CSV_START--- and ---CSV_END--- to wrap CSV contents in the response.
"""

user_prompt = """
Below is a PortQry firewall-test results log file.
Please analyze all tested ports according to the rules defined in the system prompt and generate the required output files:

1. Markdown summary.
2. CSV file passed/failed results.

Log contents:
"""

In [ ]:
messages = [
    {"role":"system", "content":system_prompt},
    {"role":"user", "content":user_prompt + log_contents}
           ] 


In [ ]:
from pathlib import Path
from datetime import datetime

openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response_text = response.choices[0].message.content
display(Markdown(response_text))

summary = response_text.split("---SUMMARY_START---")[1].split("---SUMMARY_END---")[0].strip()
csv_data = response_text.split("---CSV_START---")[1].split("---CSV_END---")[0].strip()

def write_with_backup(path: Path, content: str, encoding: str = "utf-8") -> None:
    """
    If `path` exists, rename it to a timestamped backup, then write new content.
    """
    if path.exists():
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_path = path.with_name(f"{path.stem}_{timestamp}{path.suffix}.bak")
        path.rename(backup_path)
        print(f"Backed up existing file to: {backup_path}")

    path.write_text(content, encoding=encoding)
    print(f"Wrote new file: {path}")

# current folder; change if needed
base_dir = Path(".")

md_path = base_dir / "portqry_results_summary.md"
csv_path = base_dir / "portqry_results_output.csv"

write_with_backup(md_path, summary)
write_with_backup(csv_path, csv_data)